# GET HISTORIES

In [1]:
import numpy as np
import pandas as pd
import sqlite3

import yfinance as yf

### LOAD

In [2]:
con = sqlite3.connect('market.db')

dow_table = pd.read_sql('SELECT * FROM dow_table', con)
sp5_table = pd.read_sql('SELECT * FROM sp5_table', con)
etf_table = pd.read_sql('SELECT * FROM etf_table', con)

# display(dow_table)
# display(sp5_table)
# display(etf_table)

### PREP

In [3]:
dow_symbols = dow_table['Symbol'].values
sp5_symbols = sp5_table['Symbol'].values
etf_symbols = etf_table['Symbol'].values

symbols = np.unique(np.concatenate([
    dow_symbols,
    sp5_symbols,
    etf_symbols]))

# display(symbols)

### GET

In [4]:
wide_histories = yf.download(list(symbols))

[*********************100%***********************]  604 of 604 completed


### FORMAT

In [5]:
tidy_histories = (wide_histories
    .stack()
    .reset_index()
    .rename(columns={'level_1': 'Symbol', 'Adj Close': 'Adj_Close'})
    .astype({'Date': 'datetime64[ns]', 'Volume': 'int64'}))

display(tidy_histories)

,Date,Symbol,Adj_Close,Close,High,Low,Open,Volume
0,1962-01-02,AEP,1.027578,34.312500,35.125000,34.312500,0.000000,5800
1,1962-01-02,BA,0.190931,0.823045,0.837449,0.823045,0.837449,352350
2,1962-01-02,CAT,0.495367,1.604167,1.619792,1.588542,1.604167,163200
3,1962-01-02,CNP,0.311525,10.783375,10.865333,10.783375,0.000000,13879
4,1962-01-02,CVX,0.361663,3.296131,3.296131,3.244048,0.000000,105840
...,...,...,...,...,...,...,...,...
4454354,2022-05-13,YUM,112.830002,112.830002,114.010002,111.650002,111.919998,1210603
4454355,2022-05-13,ZBH,115.070000,115.070000,116.260002,114.000000,114.269997,1118099
4454356,2022-05-13,ZBRA,336.859985,336.859985,337.750000,322.010010,322.010010,622707
4454357,2022-05-13,ZION,53.560001,53.560001,54.439999,52.849998,53.919998,1472511


### SAVE

In [6]:
tidy_histories.to_sql('tidy_histories', con, index=False, if_exists='replace')

# GET RECENT CLOSE HISTORIES

### LOAD

In [7]:
start_date = '1995-1-1'

dow_close_histories = pd.read_sql('''
    SELECT Date, Symbol, Adj_Close AS Close
    FROM tidy_histories
    WHERE Date >= ? AND Symbol IN (
        SELECT Symbol
        FROM dow_table) 
    ''', con, params=(start_date,), parse_dates=['Date'])

sp5_close_histories = pd.read_sql('''
    SELECT Date, Symbol, Adj_Close AS Close
    FROM tidy_histories
    WHERE Date >= ? AND Symbol IN (
        SELECT Symbol
        FROM sp5_table) 
    ''', con, params=(start_date,), parse_dates=['Date'])

etf_close_histories = pd.read_sql('''
    SELECT Date, Symbol, Adj_Close AS Close
    FROM tidy_histories
    WHERE Date >= ? AND Symbol IN (
        SELECT Symbol
        FROM etf_table) 
    ''', con, params=(start_date,), parse_dates=['Date'])

display(dow_close_histories)
display(sp5_close_histories)
display(etf_close_histories)

,Date,Symbol,Close
0,1995-10-02,AAPL,0.287978
1,1995-10-02,AMGN,9.392165
2,1995-10-02,AXP,8.946339
3,1995-10-02,BA,20.542841
4,1995-10-02,CAT,6.943576
...,...,...,...
188911,2022-05-13,UNH,485.399994
188912,2022-05-13,V,199.229996
188913,2022-05-13,VZ,48.180000
188914,2022-05-13,WBA,43.549999


,Date,Symbol,Close
0,1995-10-02,AAPL,0.287978
1,1995-10-02,ABC,2.491180
2,1995-10-02,ABMD,5.187500
3,1995-10-02,ABT,5.299615
4,1995-10-02,ADBE,6.301962
...,...,...,...
2899797,2022-05-13,YUM,112.830002
2899798,2022-05-13,ZBH,115.070000
2899799,2022-05-13,ZBRA,336.859985
2899800,2022-05-13,ZION,53.560001


,Date,Symbol,Close
0,1995-10-02,SPY,36.204815
1,1995-10-03,SPY,36.243706
2,1995-10-04,SPY,36.204815
3,1995-10-05,SPY,36.311749
4,1995-10-06,SPY,36.340908
...,...,...,...
384846,2022-05-13,XLY,152.320007
384847,2022-05-13,XME,50.270000
384848,2022-05-13,XOP,133.630005
384849,2022-05-13,XRT,67.720001


### SAVE

In [8]:
dow_close_histories.to_sql('dow_close_histories', con, index=False, if_exists='replace')
sp5_close_histories.to_sql('sp5_close_histories', con, index=False, if_exists='replace')
etf_close_histories.to_sql('etf_close_histories', con, index=False, if_exists='replace')